# Feature Engineering

### 1. Caricamento dei file train_data_labeled.pkl 

In [23]:
import os
import pandas as pd

path_dataframes = "dataset/dataframes"
file_path = os.path.join(path_dataframes, "train_data_labeled.pkl")

# Carica il DataFrame usando pd.read_pickle
df_train_labeled = pd.read_pickle(file_path)

In [24]:
print("Contenuto del DataFrame:")
print(df_train_labeled.head())

print("\nDimensione del DataFrame:")
print(df_train_labeled.shape)

Contenuto del DataFrame:
    TIME   P1   P2   P3   P4   P5   P6   P7  Case  Spacecraft  ...  SV3  SV4  \
0  0.000  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   
1  0.001  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   
2  0.002  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   
3  0.003  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   
4  0.004  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   

   BP1  BP2  BP3 BP4 BP5 BP6 BP7 BV1  
0   No   No   No  No  No  No  No  No  
1   No   No   No  No  No  No  No  No  
2   No   No   No  No  No  No  No  No  
3   No   No   No  No  No  No  No  No  
4   No   No   No  No  No  No  No  No  

[5 rows x 23 columns]

Dimensione del DataFrame:
(212577, 23)


Metriche nel dominio del tempo:

In [25]:
import numpy as np

def time_domain_metrics(signal):
    """
    Calcola le metriche nel dominio del tempo per un array 1D (signal).
    Restituisce un dizionario con i valori calcolati.
    """
    metrics = {}
    metrics["mean"] = np.mean(signal)
    metrics["median"] = np.median(signal)
    metrics["p25"] = np.percentile(signal, 25)
    metrics["p75"] = np.percentile(signal, 75)
    metrics["variance"] = np.var(signal)
    metrics["line_integral"] = np.trapezoid(signal)  # approssima l'integrale
    metrics["min"] = np.min(signal)
    metrics["max"] = np.max(signal)
    return metrics

Metriche nel dominio della frequenza (FFT)

In [26]:
def frequency_domain_metrics(signal, sampling_rate=1000):
    """
    Calcola alcune metriche spettrali (fft) per un array 1D (signal).
    Restituisce un dizionario con i valori calcolati.
    Puoi estendere questa funzione con SNR, SINAD, ecc.
    """
    fft_result = np.fft.fft(signal)
    freqs = np.fft.fftfreq(len(signal), d=1/sampling_rate)
    
    # Teniamo solo le frequenze positive (>0), escludendo la componente DC
    positive_indices = np.where(freqs > 0)
    fft_result = fft_result[positive_indices]
    freqs = freqs[positive_indices]
    
    power_spectrum = np.abs(fft_result) ** 2
    
    # Esempio di metriche spettrali basilari
    metrics = {}
    metrics["peak_value"] = np.max(power_spectrum)
    metrics["peak_freq"] = freqs[np.argmax(power_spectrum)]
    metrics["sum_power_spectrum"] = np.sum(power_spectrum)
    metrics["std_power_spectrum"] = np.std(power_spectrum)
    
    # (Facoltativo) Esempio di RMS nel dominio frequenziale
    metrics["rms_freq"] = np.sqrt(np.mean(power_spectrum))
    
    return metrics

Funzione per aggregare un singolo Case

In [35]:
def aggregate_case(case_group, sampling_rate=1000):
    """
    case_group: subset del DataFrame per un singolo Case
    Ritorna un dizionario con le metriche calcolate per P1..P7.
    """
    result = {}
    
    # Per le colonne che rimangono invariate all'interno dello stesso Case,
    # prendiamo il valore dalla prima riga (o dall'ultima, è uguale).
    # Aggiungile a piacimento (Spacecraft, Condition, ecc.)
    columns_to_keep = ["Spacecraft", "Condition",
                       "SV1", "SV2", "SV3", "SV4", "BP1", "BP2", "BP3", 
                       "BP4", "BP5", "BP6", "BP7", "BV1"]
    for col in columns_to_keep:
        if col in case_group.columns:
            result[col] = case_group[col].iloc[0]
    
    # Per ogni colonna di segnale (P1..P7) calcoliamo le metriche
    signal_columns = [col for col in case_group.columns 
                      if col.startswith("P") and col not in columns_to_keep]
    
    for col in signal_columns:
        signal = case_group[col].values
        
        # Calcolo metriche dominio del tempo
        td_metrics = time_domain_metrics(signal)
        # Calcolo metriche dominio della frequenza
        fd_metrics = frequency_domain_metrics(signal, sampling_rate=sampling_rate)
        
        # Inseriamo le metriche nel dizionario finale con un prefisso
        for k, v in td_metrics.items():
            result[f"{col}_time_{k}"] = v
        for k, v in fd_metrics.items():
            result[f"{col}_freq_{k}"] = v
    
    return result

Applica l’aggregazione a tutto il DataFrame

In [36]:
import pandas as pd
import os

# Raggruppa per 'Case'
grouped = df_train_labeled.groupby("Case")

# Lista dove salveremo i risultati (dizionari)
aggregated_rows = []

for case, group in grouped:
    row_dict = aggregate_case(group, sampling_rate=1000)
    # Aggiungi la colonna 'Case' in modo esplicito
    row_dict["Case"] = case
    aggregated_rows.append(row_dict)

# Costruisci il DataFrame finale
df_aggregated = pd.DataFrame(aggregated_rows)

Risultato finale

In [37]:
print("DataFrame aggregato:")
print(df_aggregated.head())


print("\nDimensioni finali:", df_aggregated.shape)

DataFrame aggregato:
   Spacecraft Condition  SV1  SV2  SV3  SV4 BP1 BP2 BP3 BP4  ...  \
0           1    Normal  100  100  100  100  No  No  No  No  ...   
1           1    Normal  100  100  100  100  No  No  No  No  ...   
2           1    Normal  100  100  100  100  No  No  No  No  ...   
3           1    Normal  100  100  100  100  No  No  No  No  ...   
4           1    Normal  100  100  100  100  No  No  No  No  ...   

  P7_time_variance P7_time_line_integral P7_time_min P7_time_max  \
0         0.329054           2366.012724   -0.003006    5.017115   
1         0.335404           2366.376508   -0.003786    4.999330   
2         0.353392           2368.725329   -0.007141    5.000936   
3         0.321221           2365.701657   -0.002395    5.008294   
4         0.324006           2366.740718   -0.002607    4.998044   

   P7_freq_peak_value  P7_freq_peak_freq  P7_freq_sum_power_spectrum  \
0        47260.680479          64.945878               237314.084554   
1        47662.86

Esportazione in .csv e .pkl

In [38]:
df_aggregated.to_csv("dataset/dataframes/train_data_aggregated.csv", index=False)
df_aggregated.to_pickle("dataset/dataframes/train_data_aggregated.pkl")